In [ ]:
# @title # **Instalar**
import os
import subprocess
from google.colab import drive
import tarfile, zipfile
from pathlib import Path

compiled = True #@param {type:"boolean"}

def extract_tar_file(tar_file_path, destination_path):
  if not Path(tar_file_path).exists():
      print(f"Tar file {tar_file_path} does not exist.")
      return

  extraction_failed = False

  with tarfile.open(tar_file_path, "r:gz") as tar:
      for member in tar.getmembers():
          try:
              tar.extract(member, destination_path)
          except Exception as e:
              print(f"Failed to extract file {member.name}: {e}")
              extraction_failed = True

  print(f"Extraction of {tar_file_path} to {destination_path} completed.")

  if os.path.exists(tar_file_path):
      try:
          os.remove(tar_file_path)
      except Exception as e:
          print(f"Failed to remove tar file {tar_file_path}: {e}")

  if extraction_failed:
      print("Extraction encountered errors.")

def install_precompiled_dependencies():
  arquivo_zip = '/content/RVCAICoverMakerDeps_lite.tar.gz'
  pasta_destino = '/content/dependencies'
  Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
  !aria2c --console-log-level=error -x 16 -s 16 -k 1M https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/RVCAICoverMakerDeps_lite.tar.gz -d /content -o RVCAICoverMakerDeps_lite.tar.gz
  source_path = "/content/RVCAICoverMakerDeps_lite.tar.gz"
  destination_path = "/"
  extract_tar_file(source_path, destination_path)

drive.mount('/content/drive')
if compiled:
  !sudo apt install aria2 -y
  install_precompiled_dependencies()
  !git clone https://github.com/blaise-tk/RVC_CLI.git
  os.chdir("/content/RVC_CLI")
  !pip install transformers==4.37.2
  !pip install tokenizers
  !python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/
  !python main.py prerequisites --pretraineds_v1 "False" --pretraineds_v2 "False" --models "True" --exe "False"
else:
  !pip install "audio-separator[gpu]" yt-dlp
  !git clone https://github.com/blaise-tk/RVC_CLI.git
  os.chdir("/content/RVC_CLI")
  pip_packages = ["fairseq>=0.12.2", "beautifulsoup4", "ffmpeg-python>=0.2.0", "numpy==1.23.5", "requests==2.31.0", "tqdm", "wget", "faiss-cpu==1.7.3", "librosa==0.9.1", "pydub==0.25.1", "pyworld==0.3.4", "praat-parselmouth==0.4.2", "resampy==0.4.2", "scipy==1.11.1", "sounddevice==0.4.6", "soundfile==0.12.1", "torchaudio==2.1.1", "praat-parselmouth", "noisereduce", "numba", "onnxruntime","onnxruntime_gpu==1.15.1","torch==2.1.1","torchcrepe==0.0.21","torchgen>=0.0.1","torch_directml","torchvision==0.16.1","einops","local-attention","matplotlib==3.7.2","tensorboard","ffmpy==0.3.1","tensorboardX","edge-tts==6.1.9","pydantic","fastapi","uvicorn"]
  subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)
  !pip install audio_upscaler pedalboard git+https://github.com/IAHispano/gdown
  !python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/
  !python main.py prerequisites --pretraineds_v1 "False" --pretraineds_v2 "False" --models "True" --exe "False"

In [ ]:
# @title # **Download do modelo**
link_modelo = "" #@param {type:"string"}
print("Baixando modelo de IA")
!python main.py download --model_link "{link_modelo}" &> /dev/null
print("Download concluido!")

In [ ]:
# @title # **Fazer o cover**
# Link do youtube da música/playlist
# @markdown ##### *Link da música/playlist*
link = "" #@param {type:"string"}
# @markdown ##### *Nome da pasta do modelo em `/content/RVC_CLI/logs`*
nome_do_modelo = "" #@param {type:"string"}
# @markdown ## **Reverb**
usar_reverb = True #@param {type:"boolean"}
reverb_size = 0.15  # @param {type:"slider", min:0.0, max:1.0, step:0.01}
reverb_wetness = 0.2 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
reverb_dryness = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
reverb_damping = 0.7 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
reverb_width = 1.0 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
# @markdown ## **Opções de mixagem do áudio**
# @markdown ### Ganho
voc_Vol = -2 # @param {type:"slider", min:-10, max:10, step:0}
inst_Vol = 0 # @param {type:"slider", min:-10, max:10, step:0}
formato_do_audio = "flac" # @param ['flac', 'mp3'] {allow-input: false}

import os
from glob import glob
from pydub import AudioSegment
from pathlib import Path
from pedalboard import Pedalboard, Reverb, HighpassFilter
from pedalboard.io import AudioFile

os.chdir("/content/RVC_CLI")

pasta_arquivos_originais = "/content/arquivos_originais"
pasta_arquivos_processados = "/content/arquivos_processados"
pasta_arquivos_vocais = "/content/arquivos_processados/arquivos_vocais"
pasta_arquivos_instrumentais = "/content/arquivos_processados/arquivos_instrumentais"
pasta_arquivos_karaoke = "/content/arquivos_processados/arquivos_karaoke"
pasta_arquivos_dereverb = "/content/arquivos_processados/arquivos_dereverb"
pasta_arquivos_deecho = "/content/arquivos_processados/arquivos_deecho"
pasta_arquivos_rvc = "/content/arquivos_processados/arquivos_rvc"
pasta_arquivos_finais = "/content/drive/MyDrive/arquivos_finais_cover"

folders = [pasta_arquivos_originais, pasta_arquivos_processados, pasta_arquivos_vocais, pasta_arquivos_instrumentais, pasta_arquivos_karaoke, pasta_arquivos_dereverb, pasta_arquivos_deecho, pasta_arquivos_rvc, pasta_arquivos_finais]

for folder in folders:
    path = Path(folder)
    if not path.exists():
        path.mkdir(parents=True, exist_ok=True)
    elif path.is_file():
        print(f"{folder} is a file, not a directory.")

def listar_arquivos(pasta):
    return os.listdir(pasta)

def buscar_arquivo_com_palavra(pasta, palavra):
    arquivos = os.listdir(pasta)
    arquivos_com_palavra = [arquivo for arquivo in arquivos if palavra in arquivo]
    if arquivos_com_palavra:
        return arquivos_com_palavra[-1]
    else:
        return None

def ultimo_arquivo_modificado(pasta):
    arquivos = [f for f in os.listdir(pasta) if os.path.isfile(os.path.join(pasta, f))]
    mais_recente = max(arquivos, key=lambda x: os.path.getmtime(os.path.join(pasta, x)))
    return mais_recente

def remover_palavra_nome_arquivo(arquivo, palavra):
    nome_arquivo = os.path.basename(arquivo)
    if palavra in nome_arquivo:
        novo_nome = nome_arquivo.replace(palavra, '')
        novo_arquivo = os.path.join(os.path.dirname(arquivo), novo_nome)
        os.rename(arquivo, novo_arquivo)
        return novo_nome
    return nome_arquivo

def find_files(directory, extensions):
  files = glob(f'{directory}/**/*{extensions}', recursive=True)
  return files[0]

def add_audio_effects(audio_path, reverb_size, reverb_wet, reverb_dry, reverb_damping, reverb_width, output_path):
    board = Pedalboard([])
    board.append(HighpassFilter())
    board.append(Reverb(room_size=reverb_size, dry_level=reverb_dry, wet_level=reverb_wet, damping=reverb_damping, width=reverb_width))
    with AudioFile(audio_path) as f:
        with AudioFile(output_path, 'w', f.samplerate, f.num_channels) as o:
            while f.tell() < f.frames:
                chunk = f.read(int(f.samplerate))
                effected = board(chunk, f.samplerate, reset=False)
                o.write(effected)
    return output_path

def juntar_arquivos_audio(input_file, vocals_path, inst_path, output_path, rvc_model_name, main_gain, inst_gain, output_format):
    main_vocal_audio = AudioSegment.from_file(vocals_path, format='flac') + main_gain
    instrumental_audio = AudioSegment.from_file(inst_path, format='flac') + inst_gain
    main_vocal_audio.overlay(instrumental_audio).export(output_path, format=output_format)
    return output_path

print("Baixando música(s)")
!yt-dlp -x -o "{pasta_arquivos_originais}/%(title)s.%(ext)s" "{link}" &> /dev/null
print("Download concluido!")
arquivos_originais = listar_arquivos("/content/arquivos_originais")

for arquivo_original in arquivos_originais:
    # Separa Vocais/Instrumentais
    print("Começando a separar os vocais e instrumentais")
    !audio-separator "{pasta_arquivos_originais}/{arquivo_original}" --log_level warning --invert_spect --normalization 1.0 -m model_bs_roformer_ep_317_sdr_12.9755.ckpt --output_dir "{pasta_arquivos_vocais}"
    arquivo_inst = buscar_arquivo_com_palavra(pasta_arquivos_vocais, "Instrumental")
    !mv "/content/arquivos_processados/arquivos_vocais/{arquivo_inst}" "/content/arquivos_processados/arquivos_instrumentais/{arquivo_inst}"
    arquivo_voc = buscar_arquivo_com_palavra(pasta_arquivos_vocais, "Vocals")
    arquivo_voc = remover_palavra_nome_arquivo(f"/content/arquivos_processados/arquivos_vocais/{arquivo_voc}", "_model_bs_roformer_ep_317_sdr_12")
    print("Vocais e instrumentais separados!")

    # Dereverb
    print("Começando a tirar o Reverb")
    !audio-separator "{pasta_arquivos_vocais}/{arquivo_voc}" --log_level warning --single_stem Instrumental  --normalization 1.0 -m Reverb_HQ_By_FoxJoy.onnx --output_dir '{pasta_arquivos_dereverb}'
    arquivo_rev = buscar_arquivo_com_palavra(pasta_arquivos_dereverb, "Instrumental")
    arquivo_rev = remover_palavra_nome_arquivo(f"/content/arquivos_processados/arquivos_dereverb/{arquivo_rev}", "_Reverb_HQ_By_FoxJoy")
    print("Reverb removido!")

    # Deecho
    print("Começando a tirar o Echo")
    !audio-separator "{pasta_arquivos_dereverb}/{arquivo_rev}" --log_level warning --normalization 1.0 -m UVR-De-Echo-Aggressive.pth --output_dir '{pasta_arquivos_deecho}'
    arquivo_no_echo = buscar_arquivo_com_palavra(pasta_arquivos_deecho, "No Echo")
    arquivo_no_echo = remover_palavra_nome_arquivo(f"/content/arquivos_processados/arquivos_deecho/{arquivo_no_echo}", "_UVR-De-Echo-Aggressive")
    arquivo_echo = buscar_arquivo_com_palavra(pasta_arquivos_deecho, "(Instrumental)_(Instrumental)")
    !rm "/content/arquivos_processados/arquivos_deecho/{arquivo_echo}"
    print("Echo removido!")

    # Karaoke
    print("Começando a tirar os Backing Vocals")
    !audio-separator "{pasta_arquivos_deecho}/{arquivo_no_echo}" --log_level warning --single_stem Instrumental  --normalization 1.0 -m UVR-BVE-4B_SN-44100-1.pth --output_dir '{pasta_arquivos_karaoke}'
    arquivo_karaoke = buscar_arquivo_com_palavra(pasta_arquivos_karaoke, "Instrumental")
    arquivo_karaoke = remover_palavra_nome_arquivo(f"/content/arquivos_processados/arquivos_karaoke/{arquivo_karaoke}", "_UVR-BVE-4B_SN-44100-1")
    print("Backing Vocals removidos!")

    # RVC
    print("Começando a inferencia do RVC")
    arquivo_pth = find_files(f"/content/RVC_CLI/logs/{nome_do_modelo}", ".pth")
    arquivo_index = find_files(f"/content/RVC_CLI/logs/{nome_do_modelo}", ".pth")
    nome_arquivo = os.path.basename(arquivo_karaoke)
    !python main.py infer --f0up_key "0" --filter_radius "3" --index_rate "0.7" --hop_length "128" --rms_mix_rate "1.0" --protect "0.3" --f0autotune "False" --f0method "rmvpe" --input_path "{pasta_arquivos_karaoke}/{arquivo_karaoke}" --output_path "/content/arquivos_processados/arquivos_rvc/{nome_arquivo}" --pth_path "{arquivo_pth}" --index_path "{arquivo_pth}" --split_audio "False" --clean_audio "False" --export_format "FLAC"  &> /dev/null
    print("Inferencia concluida!")

    # Reverb
    if usar_reverb:
      print("Aplicando Reverb")
      audio = ultimo_arquivo_modificado(pasta_arquivos_rvc)
      add_audio_effects(f"{pasta_arquivos_rvc}/{audio}", reverb_size, reverb_wetness, reverb_dryness, reverb_damping, reverb_width, f"/content/arquivos_processados/arquivos_rvc/{nome_arquivo}_output_reverb.wav")
      print("Reverb Aplicado!")

    # Junção
    print("Juntando os arquivos")
    voc = ultimo_arquivo_modificado(pasta_arquivos_rvc)
    inst = ultimo_arquivo_modificado(pasta_arquivos_instrumentais)
    arquivo_final = f"{pasta_arquivos_finais}/{nome_arquivo}"
    juntar_arquivos_audio(nome_arquivo, f"{pasta_arquivos_rvc}/{voc}", f"{pasta_arquivos_instrumentais}/{inst}", arquivo_final, nome_do_modelo, voc_Vol, inst_Vol, formato_do_audio)
    print(f"Cover de {nome_arquivo} concluido! O arquivo esta na pasta arquivos_finais_cover do seu Drive!")